In [1]:
import diagonal_b6 as b6
import pandas as pd
from tqdm import tqdm

In [2]:
w = b6.connect_insecure("0.0.0.0:8002")

In [3]:
name = "diagonal.works/edinburgh/school-crossing-analyis"

In [4]:
schools = w( b6.find(b6.tagged("#amenity", "school")) )

# TODO: Could instead be this?!
# Or, find some common building for each school?

# If so, then we can at least find the entrance ll for the school; as long as it is an area not a point.
#
# TODO: Could be some good default to do here.
#
# TODO: Investigate 'accessible'

# schools = w( b6.find(b6.tagged("#building", "school")) )

In [5]:
# w( b6.find_feature(schools[0][0]).closest_distance({"mode": "walk"}, 1000.0, b6.keyed("crossing")) )

In [12]:
results = []

# For testing, EMAS is idx 13 presently.

for school in schools[13:14]:
# for school in tqdm(schools):
    feat = w( b6.find_feature( school[0]) )
    name = feat.get_string("name")

    # entrance_ll = w( b6.entrance_approach(b6.find_feature(school[0])) )
    # print(entrance_ll)

    # TODO: A lot of this should be able to be 1 call to `w`.
    
    crossing = w( b6.find_feature(school[0]).closest({"mode": "walk"}, 1000.0, b6.keyed("crossing")) )

    speed = "unknown"
    
    try:
        # TODO: Investigate error: "ValueError: Can't convert None to proto"
        # HACK: Find the road associated to this crossing. Ideally we could do osm-style "give me the thing that contains this".
        road = w( b6.find(b6.within_cap( b6.find_feature(crossing), 1.0))
                 .filter(lambda v: b6.matches(v, b6.keyed("maxspeed"))) )
    
        if len(road) == 1:
            # TODO: Why can't I use 'map' above to remove this?
            speed = w( b6.find_feature(road[0][0]).get_string("maxspeed"))
    except:
        #print("Error computing road speed")
        speed = "error"

    dist = w( b6.find_feature(school[0]).closest_distance({"mode": "walk"}, 1000.0, b6.keyed("crossing")) )

    kind = None

    if crossing:
        kind = crossing.get_string("crossing_ref")
    
    results.append( {"Name": name, "Distance": dist, "Crossing Kind": kind, "Road speed": speed } )

In [13]:
df = pd.DataFrame(results)
df

,Name,Distance,Crossing Kind,Road speed
0,Edinburgh Montessori Arts School,400.876883,,30 mph


In [8]:
df[ df.Name.str.contains("Montessori") ]

,Name,Distance,Crossing Kind,Road speed
13,Edinburgh Montessori Arts School,400.876883,,30 mph


In [9]:
df[ (df.Distance > 400) & (df["Road speed"] == "30 mph") ]

,Name,Distance,Crossing Kind,Road speed
11,St Andrew's Fox Covert RC Primary School,436.754047,,30 mph
13,Edinburgh Montessori Arts School,400.876883,,30 mph
18,Mannafields Christian School,545.269619,,30 mph
63,Parsons Green Primary School,537.267051,,30 mph
65,Merchiston Castle School,522.102014,toucan,30 mph
80,East Craigs Primary School,805.944761,,30 mph
120,Cargilfield School,789.703537,toucan,30 mph
125,Clovenstone Primary School,450.667158,zebra,30 mph
150,Granton Primary School,816.395780,,30 mph


In [10]:
df[ (df["Road speed"] == "30 mph") ].sort_values(["Distance"])

,Name,Distance,Crossing Kind,Road speed
115,,12.312495,puffin,30 mph
52,Gracemount High School,40.730622,puffin,30 mph
12,Howdenhall & St Katharine's Special School,107.145896,,30 mph
127,Kaimes School,109.587488,puffin,30 mph
60,Portobello High School,109.712427,,30 mph
152,Holy Cross RC Primary School,116.492478,,30 mph
8,Broomhouse Primary School,152.178987,puffin,30 mph
118,St John Vianney RC Primary School,167.460627,pelican,30 mph
57,Prestonfield Primary School,175.613231,,30 mph
31,Fettes College,178.566487,,30 mph


In [11]:
len(schools)

161

## Using `accessible_all`

In [24]:
results = []

# For testing, EMAS is idx 13 presently.

# for school in schools[13:14]:
for school in tqdm(schools):
    feat = w( b6.find_feature( school[0]) )
    name = feat.get_string("name")

    is_crossing = b6.keyed("crossing") # .and_(b6.tagged("crossing_ref", "zebra"))
    
    crossings = w( b6.accessible_all([school[0]], is_crossing, 500.0, {"mode": "walk"}) )

    for _, crossing in crossings:
        speed = "unknown"
        
        try:
            # TODO: Investigate error: "ValueError: Can't convert None to proto"
            # HACK: Find the road associated to this crossing. Ideally we could do osm-style "give me the thing that contains this".
            road = w( b6.find(b6.within_cap( b6.find_feature(crossing), 1.0))
                     .filter(lambda v: b6.matches(v, b6.keyed("maxspeed"))) )
        
            if len(road) == 1:
                # TODO: Why can't I use 'map' above to remove this?
                speed = w( b6.find_feature(road[0][0]).get_string("maxspeed"))
        except:
            #print("Error computing road speed")
            speed = "error"
    
        dist = w( b6.find_feature(school[0]).closest_distance({"mode": "walk"}
                , 1000.0, b6.keyed("crossing")) )

        try:
            # Note: Skipping 'invalid//0' things here; need to find a better way to do that!
            kind = w( b6.find_feature(crossing) ).get_string("crossing_ref")
            
            results.append( {"Name": name, "Distance": dist, "Crossing Kind": kind, "Road speed": speed } )
        except:
            pass

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161/161 [11:17<00:00,  4.21s/it]


In [25]:
df = pd.DataFrame(results)
df

,Name,Distance,Crossing Kind,Road speed
0,Pirniehall Primary School,421.978393,,20 mph
1,Pirniehall Primary School,421.978393,,20 mph
2,Pirniehall Primary School,421.978393,zebra,20 mph
3,Pirniehall Primary School,421.978393,pelican,20 mph
4,Pirniehall Primary School,421.978393,pelican,20 mph
...,...,...,...,...
1051,Basil Paterson School,84.923360,zebra,20 mph
1052,Basil Paterson School,84.923360,zebra,20 mph
1053,Basil Paterson School,84.923360,,unknown
1054,Basil Paterson School,84.923360,,20 mph


In [28]:
dfi = df.set_index("Name")
dfi

,Distance,Crossing Kind,Road speed
Name,,,
Pirniehall Primary School,421.978393,,20 mph
Pirniehall Primary School,421.978393,,20 mph
Pirniehall Primary School,421.978393,zebra,20 mph
Pirniehall Primary School,421.978393,pelican,20 mph
Pirniehall Primary School,421.978393,pelican,20 mph
...,...,...,...
Basil Paterson School,84.923360,zebra,20 mph
Basil Paterson School,84.923360,zebra,20 mph
Basil Paterson School,84.923360,,unknown


In [40]:
len(set(dfi.index))

130

In [46]:
len(set(dfi[ dfi["Crossing Kind"] != "" ].index))

81

^ Most schools have at least 1 crossing.